### Notes
- probably not enough data to use word to vec

In [6]:
# Imports

# Custom imports
from utils import accessors, preprocess
from datetime import datetime as dt
import string
from tqdm.auto import tqdm
import collections

#import pandas as pd

In [2]:
show_outputs = True

In [3]:
# File and location information
path = "dataset/"
add_file = "addresses.jsonl"
cit_file = "cities.jsonl"

# Load the datasets in pandas
add_df = accessors.load_data(path, add_file) 
cit_df = accessors.load_data(path, cit_file) 

# Preprocess the datasets
add_df = preprocess.clean_strings(add_df) 
cit_df = preprocess.clean_strings(cit_df) 

if show_outputs: 
    display(add_df)
    display(cit_df)

,address,country
0,Mitterweg Angath,AT
1,"josef-franz-huter-straße 72 6020, innsbruck, a...",AT
2,"FLIRSCHBERG FLIRSCH, ÖSTERREICH",AT
3,Lochbödele Landeck Austria,AT
4,"Ainet 75 9951, Ainet",AT
...,...,...
595491,RUE DU JEU DE PAUME VILLEPREUX,FR
595492,"78955, CARRIÈRES-SOUS-POISSY",FR
595493,rue du cèdre - morainvilliers,FR
595494,"Boissets, France",FR


,city,country
0,Kappl,AT
1,Schmirn,AT
2,Zams,AT
3,Wattenberg,AT
4,Polling in Tirol,AT
...,...,...
77476,Saint-Mesmin,FR
77477,Vernoux-en-Vivarais,FR
77478,Sedze-Maubecq,FR
77479,Bulat-Pestivien,FR


### Quick data analytics

In [5]:
# Any nans in the dataset?
print(add_df.isna().sum())
print(cit_df.isna().sum())

# Number of occurences of each country 
countries = add_df.groupby('country').count()
countries.plot.barh()
countries.plot.barh(log = True)
print(len(countries))
# (TODO: stratify train/test split over this distribtion)

address    0
country    0
dtype: int64
city       0
country    0
dtype: int64


KeyboardInterrupt: 

# Simple solution 
Split address into tokens and brute force search in database 

In [7]:
t1 = dt.now()
# Split the address into a list
add_df['address_split'] = add_df['address'].str.split()

#add_df = add_df.head(20)

dupe_cities = collections.defaultdict(list)

# Loop over each row and
for i, row in tqdm(add_df.iterrows(), total=add_df.shape[0]): 
    address = row['address_split']
    for token in address: 
        if token.isnumeric(): 
            pass
        elif token in cit_df['city'].values:
            try: 
                add_df.loc[i, 'country_out'] = cit_df.loc[cit_df['city'] == token, 'country'].item()
            except: 
                dupe_cities[token].append(cit_df.loc[cit_df['city'] == token, 'country'].values)
            
display(add_df)

t2 = dt.now()
print("Time for calculation: {}".format((t2-t1)))

  0%|          | 0/595496 [00:00<?, ?it/s]

,address,country,address_split,country_out
0,mitterweg angath,AT,"[mitterweg, angath]",AT
1,joseffranzhuterstraße 72 6020 innsbruck austria,AT,"[joseffranzhuterstraße, 72, 6020, innsbruck, a...",AT
2,flirschberg flirsch österreich,AT,"[flirschberg, flirsch, österreich]",AT
3,lochbödele landeck austria,AT,"[lochbödele, landeck, austria]",AT
4,ainet 75 9951 ainet,AT,"[ainet, 75, 9951, ainet]",AT
...,...,...,...,...
595491,rue du jeu de paume villepreux,FR,"[rue, du, jeu, de, paume, villepreux]",FR
595492,78955 carrièressouspoissy,FR,"[78955, carrièressouspoissy]",FR
595493,rue du cèdre morainvilliers,FR,"[rue, du, cèdre, morainvilliers]",FR
595494,boissets france,FR,"[boissets, france]",PT


Time for calculation: 1:54:44.345351


In [64]:
def accuracy(df):

# A column of correctly labelled
add_df['correct'] = add_df['country'] == add_df['country_out']

# Count number of corrects for each country
results = add_df.groupby('country').agg(correct=('correct','sum'),total=('correct','count')).reset_index()
results['accuracy'] = results['correct']/results['total']
display(results)


,country,correct,total,accuracy
0,AT,1523,1884,0.808386
1,BE,36375,39338,0.924678
2,CZ,16044,19110,0.839560
3,DE,45910,52647,0.872034
4,EE,0,11639,0.000000
5,FR,173988,257025,0.676930
6,IT,27071,33093,0.818028
7,LU,1519,1591,0.954745
8,NL,86107,90589,0.950524
9,PL,37743,40658,0.928304


### Short-comings
- not all addresses are following the same format, e.g. some have comma's, some do not. So end up with useless characters in your lists. Note, have now removed punctuation in preprocessing
- not all cities are one single string 
- special characters might change (e.g. ß or ss) 
- some cities exist in multiple countries